In [95]:
import tensorflow as tf

fashion=tf.keras.datasets.fashion_mnist

(x_train, y_train), (x_test, y_test)=fashion.load_data()

print (x_train.shape)
print (y_train.shape)

print (x_test.shape)
print (y_test.shape)

(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


In [96]:
len(set(y_train))

10

In [97]:
print(x_test[0])

[[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   3   1   0   0   7   0  37   0   0]
 [  0   0   0   0   0   0   0   0   0   0   0   0   0   1   2   

In [98]:
BATCH_SIZE=128
EPOCH=50
cls_num=len(set(y_train))
image_shape=(28,28)

In [99]:
def create_model(features,feature_columns,output_cls):
    
    input_layer=tf.feature_column.input_layer(features,feature_columns)
    
    net = tf.reshape(input_layer, [-1, 28, 28, 1]) 
    
#     tf.layers.conv2d   isdeprecated ,keras.layers.conv2d
    conv1=tf.keras.layers.Conv2D(filters=16,kernel_size=[3,3],activation='relu',name='conv1')(net)
    pool1=tf.keras.layers.MaxPool2D(pool_size=(3,3))(conv1)
    conv2=tf.keras.layers.Conv2D(filters=8,kernel_size=[5,5],activation='relu',name='conv2')(pool1)
    flat=tf.keras.layers.Flatten()(conv2)
    dens1=tf.keras.layers.Dense(units=128,activation='relu',name='dense1')(flat)
    logits=tf.keras.layers.Dense(units=output_cls,name='dense_output')(dens1)
    return logits
def  model_fn_builder(lr):
    # 该方法实际 创建 estimator的model_fn
    # 可以 有其他操作
    def model_fn(features, labels, mode, params,config): #estimator需要的model_fn 参数固定
        '''
        features: from input_fn的返回  切记返回的顺序
        labels： from input_fn 的返回  切记返回的顺序
        mode: tf.estimator.ModeKeys实例的一种
        params: 在初始化estimator时 传入的参数列表，dict形式,或者直接使用self.params也可以
        config:初始化estimator时 的 Runconfig

        '''
        logits=create_model(features,params['feature_columns'],params['output_cls'])
        
        pre_cls=tf.math.argmax(input=logits,axis=1)
        pre_prob=tf.nn.softmax(logits=logits,axis=1)
        
        is_predict=mode==tf.estimator.ModeKeys.PREDICT
        if not is_predict:
            # train .eval
            loss=tf.losses.sparse_softmax_cross_entropy(labels=tf.cast(labels,tf.int32),logits=logits)

            def metric_fn(labels,predictions):
                '''
                define metrics
                '''
                accuracy,accuracy_update=tf.metrics.accuracy(labels=labels,predictions=predictions,name='image_accuracy')
    #                 auc=tf.metrics.auc(labels=labels,predictions=predictions,name='iris_auc')
                recall,recall_update=tf.metrics.recall(labels=labels,predictions=predictions,name='image_recall')
                precision,precision_update=tf.metrics.precision(labels=labels,predictions=predictions,name='image_precision')
    #                 with tf.control_dependencies([recall,precision]):
    #                     f1_score=f1(recall=recall,precision=precision)

                return {
                    'accuracy':(accuracy,accuracy_update),
                    'recall':(recall,recall_update),
                    'precision':(precision,precision_update)                  
                }


            if mode==tf.estimator.ModeKeys.EVAL:
                return tf.estimator.EstimatorSpec(mode=mode,loss=loss,eval_metric_ops=metric_fn(labels,pre_cls))

            # train process
            train_op=tf.train.AdamOptimizer(learning_rate=lr).minimize(loss=loss,global_step=tf.train.get_global_step())
            return tf.estimator.EstimatorSpec(mode=mode,loss=loss,train_op=train_op,eval_metric_ops=metric_fn(labels,pre_cls))


        else:
            predictions={'predict_cls':pre_cls,'predict_pro':pre_prob}
            return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)     
    return model_fn
      

In [100]:
def input_fn_builder(x,y,batch_size,epochs,is_train=True):
    '''
    创建 输入函数闭包
    '''
    # 可以执行其它操作
    
    def input_fn():
        dataset=tf.data.Dataset.from_tensor_slices(({'images':x},y) )  
        if is_train:
            dataset=dataset.shuffle(1000).repeat(epochs)
        dataset=dataset.batch(batch_size)
        return dataset # 返回的 顺序要和 model_fn一致 或者 dataset元素 格式为（features,label）元组 也可以
    return input_fn

In [101]:
model_dir=r'F:\testDemo\AI\estimator\model\fashion'
params={}
feature_columns=[tf.feature_column.numeric_column('images',shape=image_shape)]

output_cls=cls_num
params['feature_columns']=feature_columns
params['output_cls']=output_cls


In [102]:
feature_columns

[NumericColumn(key='images', shape=(28, 28), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [103]:
# tf.summary.FileWriterCache.clear()  # 该步骤 只有当您 对 model_dir进行过删除 又重建（相同路径），导致没有events生产时，再添加
config=tf.estimator.RunConfig(save_checkpoints_steps=100)

estimator=tf.estimator.Estimator(model_fn=model_fn_builder(0.001),model_dir=model_dir,params=params,config=config)

INFO:tensorflow:Using config: {'_model_dir': 'F:\\testDemo\\AI\\estimator\\model\\fashion', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 100, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002AC8409FE80>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [104]:
train=estimator.train(input_fn=input_fn_builder(x=x_train,y=y_train,batch_size=BATCH_SIZE,epochs=EPOCH,is_train=True),steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from F:\testDemo\AI\estimator\model\fashion\model.ckpt-20000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 20000 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:loss = 0.14470702, step = 20001
INFO:tensorflow:Saving checkpoints for 20100 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 21.2633
INFO:tensorflow:loss = 0.20766848, step = 20101 (4.705 sec)
INFO:tensorflow:Saving checkpoints for 20200 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 28.3848
INFO:tensorflow:loss = 0.19923146, step = 20201 (3.523 sec)
INFO:tensorflow:Saving checkpoints for 20300 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensor

INFO:tensorflow:global_step/sec: 20.8501
INFO:tensorflow:loss = 0.19209112, step = 23901 (4.798 sec)
INFO:tensorflow:Saving checkpoints for 24000 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 25.7945
INFO:tensorflow:loss = 0.17386952, step = 24001 (3.875 sec)
INFO:tensorflow:Saving checkpoints for 24100 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 26.2126
INFO:tensorflow:loss = 0.122896485, step = 24101 (3.815 sec)
INFO:tensorflow:Saving checkpoints for 24200 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 24.7605
INFO:tensorflow:loss = 0.20588404, step = 24201 (4.039 sec)
INFO:tensorflow:Saving checkpoints for 24300 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 23.0314
INFO:tensorflow:loss = 0.12184756, step = 24301 (4.342 sec)
INFO:tensorflow:Saving checkpoints for 24400 into F:\testDemo\AI\estimator\model\fashion\m

INFO:tensorflow:global_step/sec: 25.3982
INFO:tensorflow:loss = 0.12389478, step = 28001 (3.938 sec)
INFO:tensorflow:Saving checkpoints for 28100 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 23.1332
INFO:tensorflow:loss = 0.12654938, step = 28101 (4.324 sec)
INFO:tensorflow:Saving checkpoints for 28200 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 27.4123
INFO:tensorflow:loss = 0.16295114, step = 28201 (3.646 sec)
INFO:tensorflow:Saving checkpoints for 28300 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 27.3025
INFO:tensorflow:loss = 0.08963808, step = 28301 (3.665 sec)
INFO:tensorflow:Saving checkpoints for 28400 into F:\testDemo\AI\estimator\model\fashion\model.ckpt.
INFO:tensorflow:global_step/sec: 25.3233
INFO:tensorflow:loss = 0.14725712, step = 28401 (3.947 sec)
INFO:tensorflow:Saving checkpoints for 28500 into F:\testDemo\AI\estimator\model\fashion\mo

In [106]:
estimator.evaluate(input_fn=input_fn_builder(x=x_test,y=y_test,batch_size=BATCH_SIZE,epochs=EPOCH,is_train=False))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-06-06T06:50:05Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from F:\testDemo\AI\estimator\model\fashion\model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-06-06-06:50:06
INFO:tensorflow:Saving dict for global step 30000: accuracy = 0.8644, global_step = 30000, loss = 0.7665193, precision = 0.9817285, recall = 0.9731111
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 30000: F:\testDemo\AI\estimator\model\fashion\model.ckpt-30000


{'accuracy': 0.8644,
 'loss': 0.7665193,
 'precision': 0.9817285,
 'recall': 0.9731111,
 'global_step': 30000}

In [107]:
def predict_fn_builder(x,batch_size):
    '''
    创建 输入函数闭包
    '''
    # 可以执行其它操作
    
    def input_fn():
        dataset=tf.data.Dataset.from_tensor_slices(({'images':x}) )  

        dataset=dataset.batch(batch_size)
        return dataset # 返回的 顺序要和 model_fn一致 或者 dataset元素 格式为（features,label）元组 也可以
    return input_fn

In [108]:
pre_dataset=x_train[0:10]
pre_y=y_train[0:10]
pre_y

array([9, 0, 0, 3, 0, 2, 7, 2, 5, 5], dtype=uint8)

In [109]:
predictions=estimator.predict(input_fn=predict_fn_builder(x=x_train[0:10],batch_size=10))

In [110]:
for p in predictions:
    print(p)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from F:\testDemo\AI\estimator\model\fashion\model.ckpt-30000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'predict_cls': 9, 'predict_pro': array([0.0000000e+00, 9.7136024e-32, 4.8922935e-37, 2.4414274e-34,
       2.6000109e-38, 5.0287287e-18, 7.1402950e-31, 8.2528209e-07,
       3.1106856e-26, 9.9999917e-01], dtype=float32)}
{'predict_cls': 0, 'predict_pro': array([9.9999535e-01, 4.0889493e-34, 2.9028916e-15, 6.8556294e-09,
       3.9609319e-15, 8.1493402e-24, 4.6315395e-06, 2.0719370e-12,
       1.2751717e-27, 3.6395057e-27], dtype=float32)}
{'predict_cls': 0, 'predict_pro': array([9.9918240e-01, 5.6884212e-14, 6.2230316e-07, 6.4789201e-05,
       2.0265060e-10, 3.8250029e-14, 7.5217051e-04, 1.1419887e-18,
       4.0469545e-19, 1.0197527e-30], dtype=float32)}
{'predict_cls': 6, 'predict_pro': array([8.

 save as image

In [111]:
import numpy as np
from PIL import Image, ImageOps
import os
def save_image(filename, data_array):
    im = Image.fromarray(data_array.astype('uint8'))
    im_invert = ImageOps.invert(im)
    im_invert.save(filename)

i=0
for item in x_train:
    i+=1
    filename='{}/{}.jpg'.format('data/fashion/images',str(i))    
    save_image(filename=filename,data_array=item)
    if i==10:
        break
    

In [114]:

def serving_input_receiver_fn():
    def decode_and_resize(image_str_tensor):
        """Decodes jpeg string, resizes it and returns a uint8 tensor."""
        image = tf.image.decode_jpeg(image_str_tensor, channels=1) # 在此处 是 使用灰度图，所以channel=1
        image = tf.expand_dims(image, 0)
        image = tf.image.resize_bilinear(
             image, [28, 28], align_corners=False)# 根据您 自己图的 大小设置 height width
        image = tf.squeeze(image, squeeze_dims=[0])
        image = tf.cast(image, dtype=tf.uint8)
        return image

     # Optional; currently necessary for batch prediction.
    key_input = tf.placeholder(tf.string, shape=[None]) 
    key_output = tf.identity(key_input)

    input_ph = tf.placeholder(tf.string, shape=[None], name='image_binary')
    images_tensor = tf.map_fn(
      decode_and_resize, input_ph, back_prop=False, dtype=tf.uint8)
    receiver_tensors = {'image_bytes': input_ph}
   
    features = {
       'images': images_tensor
    }    
    return tf.estimator.export.ServingInputReceiver(features,receiver_tensors)

estimator.export_savedmodel('export_base/fashion',serving_input_receiver_fn=serving_input_receiver_fn)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Restoring parameters from F:\testDemo\AI\estimator\model\fashion\model.ckpt-30000
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export_base/fashion\temp-b'1559803909'\saved_model.pb


b'export_base/fashion\\1559803909'

In [ ]:
# 请求体
{"instances" : [{"image_bytes": {"b64":"/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/APf6yfEuvW/hnw/d6vcjdHbpkIDguScAD6k1heHfiVomuQKZJhbS7QW3/cz35H3focV0Nrr2l3sbSQXsTKrbTk45xnofYg/jWlmvHPjzrdk2g2OjxTwTXTXnnSRI+541RGGSB0yWHXHtnnHhayiFo3t1khkAHIYg59RzxX0L8FkvP+EGkmt54JhPeySO0xO8PtVWBx15XgntivNfGupeMB4o1K2vdVv4P3zmO3ErpF5efl2qDgjGORn35rkoFWCYm4tXuEZCH2Hy2Vuu5WIOCD6gggkHrkRi0juHdIYliV+WWWcuWx0zgBf0r2X4O319o/hO8t4dMmu0bUHfzIVLLnZGMZ/CvXNR0nTtXgEGpWFreRA5CXESyAH1AI4NZUPgPwjbvvj8M6SG9TaIf5irY8LeHl6aDpY+lnH/AIVqRxRwxrHEioi8BVGAPwr/2Q=="}},
{"image_bytes": {"b64":"/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBwcJCQgKDBQNDAsLDBkSEw8UHRofHh0aHBwgJC4nICIsIxwcKDcpLDAxNDQ0Hyc5PTgyPC4zNDL/wAALCAAcABwBAREA/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/9oACAEBAAA/AO9+LPjO88KaPZQabKsN9fyMFlKhikaAbiAeM5ZRznqa8eitvEWqLFqk3iuG2luot6NdazLDM0YlMY4UH5d+QBTRqXibweltf2HiSOeC4eQRtZ37XMLum3cHVhjd868kZ5619K6FqkeuaDp+qRLtS7t0m2/3SwBI/A8fhXN+Ovh3YeNfLuLi8vYLm3iZIfJdSnPPKsD3x0x0FfP9l4r8R6XZQWdlrFxbQQAiOONI8JnJOMqT1LdT3rd8LQXvxJ8U2+meItXvZ4I45ZwyeWrrwo4wuOcLnIPAr6G0HRbXw7oltpNk0rW9upCGVtzHJJOT9Sa0a+MXbnJJOMDmvQPgy23x/D/tWsyfop/pX0dRXy5/witnO+Tc3S7iSQpT1Pqtdr8NPDltpnjG1uIri5dhFLxIUxyuOyivbq//2Q=="}}
]}
# 返回
{
    "predictions": [
        {
            "predict_pro": [
                0.0011323,
                0.000574304,
                0.00000434548,
                0.0000422241,
                1.86852e-8,
                3.08857e-13,
                0.00000731479,
                5.13255e-9,
                0.99824,
                1.81437e-10
            ],
            "predict_cls": 8
        },
        {
            "predict_pro": [
                7.66782e-8,
                4.37143e-13,
                0.000230125,
                3.37866e-9,
                0.998791,
                7.81147e-10,
                0.000125866,
                1.52834e-8,
                0.00085307,
                1.49522e-16
            ],
            "predict_cls": 4
        }
    ]
}
